## JIT Compilation in Julia

A discussion of just-in-time compilation in Julia and how it affects performance.  

Prepared for WAMS 2017 by [John Stachurski](http://johnstachurski.net/)

### Paradigms

* AOT compilation
* Interpreters
* JIT compilation

### How does JIT compilation work?

Here's a simple function.

In [1]:
f(x, y) = 2x + y

f (generic function with 1 method)

When a computer executes this, it needs to know the type of `x` and `y`

* floating point addition?
* integer addition?

AOT compiler approach: 

* Insist that the types are specified in the function declaration
* Generate compiled code accordingly

Interpreted approach: 

* Figure it what `+` means at run time by checking types
* perform action

JIT compiler approach: 

* Inspect types of the parameters `x` and `y` when the function is called
* Build an appropriate compiled version

* For subsequent calls `f(x, y)`, test the type of `x` and `y` and 
    * dispatch to existing compiled version if same as before, or
    * compile a new version, if different

#### Example

First time we call, Julia compiles `f` to native machine code and runs it.

In [3]:
f(1, 1)

3

In [4]:
@code_native f(1, 1)

	.text
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	(%rsi,%rdi,2), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


When we call it with different argument times, it compiles a new version.

In [5]:
f(1.0, 1.0)

3.0

In [6]:
@code_native f(1.0, 1.0)

	.text
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	addsd	%xmm0, %xmm0
	addsd	%xmm1, %xmm0
	popq	%rbp
	retq
	nop


Type inference is performed on the fly:

In [7]:
function g(x)
    a = 2x
    b = 3a
    return b
end

g (generic function with 1 method)

In [8]:
g(1)

6

In [9]:
@code_native g(1)

	.text
Filename: In[7]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	addq	%rdi, %rdi
	leaq	(%rdi,%rdi,2), %rax
Source line: 4
	popq	%rbp
	retq
	nopl	(%rax)


In [10]:
g(1.0)

6.0

In [11]:
@code_native g(1.0)

	.text
Filename: In[7]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	addsd	%xmm0, %xmm0
	movabsq	$140283090539048, %rax  # imm = 0x7F9633C79228
Source line: 3
	mulsd	(%rax), %xmm0
Source line: 4
	popq	%rbp
	retq
	nopl	(%rax,%rax)


We can specify the argument type if we want.

In [12]:
function g_typed(x::Float64)
    a = 2x
    b = 3a
    return b
end

g_typed (generic function with 1 method)

In [13]:
g_typed(1.0)

6.0

But it doesn't affect speed, at least in theory, since type inference occurs when the type isn't specified.  On the other hand, it allows us to write specialized code for different arguments.

In [14]:
function h(x::Float64)
    println("Doing something with a float")
end

h (generic function with 1 method)

In [15]:
function h(x::Int64)
    println("Doing something with an integer")
end

h (generic function with 2 methods)

In [16]:
h(1.0)

Doing something with a float


In [17]:
h(1)

Doing something with an integer


### Julia's JIT compiler is fast.

In [18]:
function quad(x0, n)
    x = x0
    for i in 1:n
        x = 4.0 * x * (1.0 - x)
    end
    return x
end

quad (generic function with 1 method)

In [19]:
n = 10_000_000

10000000

In [20]:
@time x = quad(0.2, n)

  0.037311 seconds (1.68 k allocations: 98.059 KiB)


0.9942245057972676

In [21]:
@time x = quad(0.2, n)

  0.035310 seconds (5 allocations: 176 bytes)


0.9942245057972676

Faster than comparable C code on my machine.   The reason is that the JIT compiler generates nice, efficient machine code.

In [22]:
@code_native quad(0.2, 10)

	.text
Filename: In[18]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	testq	%rdi, %rdi
	jle	L69
	movabsq	$140283090540376, %rax  # imm = 0x7F9633C79758
	movsd	(%rax), %xmm1           # xmm1 = mem[0],zero
	movabsq	$140283090540384, %rax  # imm = 0x7F9633C79760
Source line: 4
	movsd	(%rax), %xmm2           # xmm2 = mem[0],zero
	nopw	%cs:(%rax,%rax)
L48:
	movapd	%xmm2, %xmm3
	subsd	%xmm0, %xmm3
	mulsd	%xmm1, %xmm0
	mulsd	%xmm3, %xmm0
Source line: 3
	decq	%rdi
	jne	L48
Source line: 6
L69:
	popq	%rbp
	retq
	nopw	(%rax,%rax)


### JIT compilers can fail to produce fast code, though

In [23]:
b = 1.0

function g_with_global(a)
    for i in 1:10_000_000
        tmp = a + b
    end
end

g_with_global (generic function with 1 method)

In [24]:
@time g_with_global(1.0)

  0.231241 seconds (20.00 M allocations: 305.227 MiB, 7.54% gc time)


In [25]:
function g_without_global(a)
    b = 1.0
    for i in 1:10_000_000
        tmp = a + b
    end
end

g_without_global (generic function with 1 method)

In [27]:
@time g_without_global(1.0)

  0.000002 seconds (4 allocations: 160 bytes)


Let's have a look at the machine code.

In [28]:
@code_native g_with_global(1.0)

	.text
Filename: In[23]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$56, %rsp
	movsd	%xmm0, -48(%rbp)
	movq	%fs:0, %r12
	addq	$-10888, %r12           # imm = 0xD578
	xorpd	%xmm0, %xmm0
	movupd	%xmm0, -72(%rbp)
	movq	$0, -56(%rbp)
	movq	$6, -88(%rbp)
	movq	(%r12), %rax
	movq	%rax, -80(%rbp)
	leaq	-88(%rbp), %rax
	movq	%rax, (%r12)
	movabsq	$140283032602472, %rax  # imm = 0x7F9630538768
	movl	$10000000, %ebx         # imm = 0x989680
Source line: 5
	leaq	457174864(%rax), %r13
	leaq	457158408(%rax), %r15
	movabsq	$jl_apply_generic, %r14
	nopw	%cs:(%rax,%rax)
L128:
	movabsq	$140283032602472, %rax  # imm = 0x7F9630538768
	movq	(%rax), %rax
	movq	%rax, -56(%rbp)
	movq	%r13, -72(%rbp)
	movl	$1432, %esi             # imm = 0x598
	movl	$16, %edx
	movq	%r12, %rdi
	movabsq	$jl_gc_pool_alloc, %rax
	callq	*%rax
	movq	%r15, -8(%rax)
	movsd	-48(%rbp), %xmm0        # xmm0 = mem[0],zero
	movsd	%xmm0, (%rax)
	movq	%rax, -64(%rbp)
	movl	$3, %esi
	leaq	-72

In [29]:
@code_native g_without_global(1.0)

	.text
Filename: In[25]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 4
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


Although I can fix this using `const`

In [30]:
const b_const = 1.0

1.0

In [31]:
function g_with_const_global(a)
    for i in 1:10_000_000
        tmp = a + b_const
    end
end

g_with_const_global (generic function with 1 method)

In [32]:
@code_native g_with_const_global(1.0)

	.text
Filename: In[31]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [33]:
@time g_with_const_global(1.0)

  0.000053 seconds (5 allocations: 240 bytes)


### Further Information

* https://lectures.quantecon.org/jl/types_methods.html
* https://lectures.quantecon.org/jl/need_for_speed.html